In [1]:
from scipy.signal import find_peaks, peak_prominences
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.rcParams['figure.figsize'] = [15, 20]
import gc
import numpy as np
from morning.pipeline.stream.cybos.stock.db.min_excel_tick import MinExcelTick
from morning.pipeline.converter.cybos.stock.day_tick import StockDayTickConverter
import pandas as pd
from datetime import date
from morning.pipeline.stream.cybos.stock.db.min_tick import MinTick
from morning.pipeline.chooser.cybos.db.kosdaq_all_chooser import KosdaqAllChooser
from morning.pipeline.chooser.cybos.db.kosdaq_bull_chooser import KosdaqBullChooser
from morning.back_data.fetch_stock_data import get_day_minute_period_data
from morning.back_data.holidays import is_holidays
from morning.pipeline.chooser.cybos.db.kosdaq_search_bull_chooser import KosdaqSearchBullChooser

from datetime import timedelta, datetime

In [2]:
def find_trend(x, indexes, prominences, is_upper):
    
    if not is_upper and indexes.shape[0] < 3:
        return False, False
    elif is_upper and indexes.shape[0] < 3:
        return False, True
    
    short_trend = x[indexes[-1]] - x[indexes[-2]] > 0
    for i, v1 in enumerate(reversed(indexes)):
        for v2 in reversed(indexes[:indexes.shape[0] - 1 - i]):
            found = True
            for c in indexes:
                if v1 == c or v2 == c: continue
                x1, x2, y1, y2, cx, cy = v1, v2, x[v1], x[v2], c, x[c]
                result = (y1 - y2) * cx + (x2 - x1) * cy + x1 * y2 - x2 * y1

                if (is_upper and result < 0) or (not is_upper and result > 0):
                    found = False
                    break
            
            if found:
                return short_trend, x[v1] - x[v2] > 0
    return short_trend, False

In [3]:
class Receiver:
    def __init__(self):
        self.data = []
    
    def received(self, datas):
        for d in datas:
            self.data.append(d)

def get_reversed(s):
    distance_from_mean = s.mean() - s
    return distance_from_mean + s.mean()

In [4]:
def calculate(df, is_rev, is_debug=False):
    x = None
    if not is_rev:
        x = df['avg']
    else:
        x = df['rev']

    peaks, _ = find_peaks(x, distance=10)
    prominences = peak_prominences(x, peaks)[0]

    peaks = np.extract(prominences > x.mean() * 0.003, peaks)
    prominences = np.extract(prominences > x.mean() * 0.003, prominences)
    peaks = np.r_[np.array([0]), peaks]

    x = df['avg']
    utrend = find_trend(x, peaks, prominences, not is_rev)

    return peaks, prominences, utrend

In [5]:
def handle_graph(data, buy_points, sell_points, code, today, profits, risks):

    df = pd.DataFrame(data)

    df['buy1'] = False
    df['sell1'] = False

    for b1 in buy_points:
        df.loc[df['date'] == b1, 'buy1'] = True

    for s1 in sell_points:
        df.loc[df['date'] == s1, 'sell1'] = True
    #print(buy_points)

    df['avg'] = df['close_price'].rolling(10).mean()
    not_non = df['avg'].notna().idxmax()
    if not_non > 0:
        df['avg'] = df['avg'].fillna(df['avg'].iloc[not_non])
    price = np.array(df['avg'])
    reverse_price = np.array(get_reversed(price))
    df['rev'] = reverse_price
    x = df['avg']

    peaks, prominences, utrend = calculate(df, False)
    peaks_b, prominences_b, utrend_b = calculate(df, True)

    fig, axs = plt.subplots(2)

    axs[0].plot(x)
    axs[0].plot(df['close_price'])
    axs[0].plot(df[df['buy1'] == True].index.values, df.loc[df['buy1'] == True, 'avg'], 'o', color='black', markersize=6)
    axs[0].plot(df[df['sell1'] == True].index.values, df.loc[df['sell1'] == True, 'avg'], 'D', color='red', markersize=6)
    axs[0].plot(peaks, x[peaks], 'x', color='red')
    axs[0].plot(peaks_b, x[peaks_b], 'x', color='blue')
    index_v = df[df['sell1'] == True].index.values
    avg_v = df.loc[df['sell1'] == True, 'avg']
    
    risk_index = df[df['buy1'] == True].index.values
    risk_v = df.loc[df['buy1'] == True, 'avg']
    for i, p in enumerate(profits):
        axs[0].annotate(str(p), (index_v[i], avg_v.iloc[i] * 1.003), fontsize=15)
        
    for i, p in enumerate(risks):
        axs[0].annotate(str(p), (risk_index[i], risk_v.iloc[i] * 1.003), fontsize=15, color='red')
        
    axs[1].plot(df['cum_buy_volume'], color='red')
    axs[1].plot(df['cum_sell_volume'], color='blue')
    plt.savefig(today.strftime('%Y%m%d_') + code + '.png', dpi=300)
    axs[0].cla()
    axs[1].cla()
    fig.clf()
    plt.close('all')
    gc.collect()


In [6]:
def handle(code, today):
    sdtc = StockDayTickConverter()
    gdmpd = get_day_minute_period_data(code, today, today)
    met = MinTick(today)
    met.set_target('cybos:' + code)
    met.set_output(sdtc)
    rcv = Receiver()
    sdtc.set_output(rcv)
    open_price = 0
    stage = 0
    buy_prices = [0, 0]
    buy_points = []
    sell_points = []
    risks = []
    buy_hold = None
    profits = []
    long_price = 0
    last_time = None
    
    last_price = 0
    while met.received(None) > 0:
        data = rcv.data

        if open_price == 0:
            open_price = data[-1]['start_price']

        if len(data) < 10: continue

        df = pd.DataFrame(data)

        if buy_hold is not None and df['date'].iloc[-1] - buy_hold < timedelta(minutes=10):
            continue

        last_price = data[-1]['close_price']
        last_time = data[-1]['date']
        
        df['avg'] = df['close_price'].rolling(10).mean()
        not_non = df['avg'].notna().idxmax()
        if not_non > 0:
            df['avg'] = df['avg'].fillna(open_price)

        price = np.array(df['avg'])
        reverse_price = np.array(get_reversed(price))
        df['rev'] = reverse_price
        peaks, prominences, utrend = calculate(df, False)
        peaks_b, prominences_b, utrend_b = calculate(df, True)

        bottom_trend = utrend_b[0] and utrend_b[1]
        roof_trend = utrend[0] and utrend[1]
        cum_sum = (data[-1]['cum_buy_volume'] + data[-1]['cum_sell_volume']) * 0.1
        volume_trend = data[-1]['cum_buy_volume'] > data[-1]['cum_sell_volume'] + cum_sum
        """
        if df['date'].iloc[-1] > datetime(2019, 12, 6, 13, 6):
            print('utrend', utrend_b[0], utrend_b[1])
            calculate(df, True, True)
            break
        """
        if stage == 0:
            bottom_trend = utrend_b[0] and utrend_b[1]
            roof_trend = utrend[0] and utrend[1]
            volume_trend = data[-1]['cum_buy_volume'] > data[-1]['cum_sell_volume']
            price_exceed = (data[-1]['close_price'] - open_price) / open_price * 100. > 15.
            if bottom_trend and not roof_trend and volume_trend and not price_exceed:
                stage = 1
                buy_prices[0] = df['avg'].iloc[peaks[-1]]
                if len(peaks) > 1:
                    buy_prices[1] = df['avg'].iloc[peaks[-2]]
                else:
                    buy_prices[1] = df['avg'].iloc[peaks[-1]]
                #print('Go S1', df['date'].iloc[-1], buy_prices)
                
        elif stage == 1:
            #print(df['date'].iloc[-1], utrend_b[0], utrend_b[1])
            if not volume_trend or not bottom_trend:
                if buy_hold is not None:
                    print(code, 'SELL', df['date'].iloc[-1], df['close_price'].iloc[-1])
                    sell_points.append(df['date'].iloc[-1])
                    f = float("{0:.2f}".format((df['close_price'].iloc[-1] - long_price) / long_price * 100.))
                    profits.append(f)
                    buy_hold = None
                stage = 0
            else:
                if df['close_price'].count() > 50:
                    start_price = df['close_price'].iloc[-50:].min()
                    close_price = df['close_price'].iloc[-50:].max()
                else:
                    start_price = df['close_price'].min()
                    close_price = df['close_price'].max()
                    
                risk = float("{0:.2f}".format((close_price - start_price) / start_price * 100.))
                
                if buy_hold is None and (df['avg'].iloc[-1] > buy_prices[0] or df['avg'].iloc[-1] > buy_prices[1]):
                    buy_points.append(df['date'].iloc[-1])
                    buy_hold = df['date'].iloc[-1]    
                    
                    print(code, 'BUY1', df['date'].iloc[-1], df['close_price'].iloc[-1], 'RISK', risk)
                    risks.append(risk)
                    long_price =  df['close_price'].iloc[-1]
    if buy_hold is not None:
        print(code, 'SELL', last_price)
        sell_points.append(last_time)
        f = float("{0:.2f}".format((last_price - long_price) / long_price * 100.))
        profits.append(f)
        
    if len(buy_points) > 0:
        handle_graph(rcv.data, buy_points, sell_points, code, today, profits, risks)
    
    
    return profits

In [7]:
from datetime import date
from morning.back_data.holidays import is_holidays

from_datetime = datetime(2018, 1, 1)
profits = []
profit_records = []
while from_datetime < datetime(2019, 12, 7):
    if is_holidays(from_datetime):
        from_datetime += timedelta(days = 1)
        continue
    kbc = KosdaqSearchBullChooser(from_datetime.date())
    if len(kbc.codes) == 0:
        from_datetime += timedelta(days = 1)
        continue
    print('START', from_datetime)
    codes = kbc.codes

    for code in codes:
        p = handle(code, from_datetime.date())
        profits.extend(p)
        print('CURRENT:', sum(profits))
        if len(p) > 0:
            profit_records.append((from_datetime, code, p))
    from_datetime += timedelta(days = 1)

2018-01-02
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
START 2018-01-02 00:00:00
Length 321
A215600 BUY1 2018-01-02 11:40:00 98400 RISK 2.82
A215600 SELL 2018-01-02 12:00:00 97800
CURRENT: -0.61
Length 318
CURRENT: -0.61
Length 321
CURRENT: -0.61
Length 319
A058820 BUY

CURRENT: 64.45000000000002
Length 380
A139050 BUY1 2018-01-04 13:14:00 5260 RISK 9.58
A139050 SELL 2018-01-04 13:25:00 5070
CURRENT: 60.84000000000002
Length 381
CURRENT: 60.84000000000002
Length 377
CURRENT: 60.84000000000002
Length 381
CURRENT: 60.84000000000002
Length 381
CURRENT: 60.84000000000002
Length 378
A018700 BUY1 2018-01-04 10:09:00 4170 RISK 12.4
A018700 SELL 2018-01-04 10:56:00 4490
CURRENT: 68.51000000000002
Length 381
CURRENT: 68.51000000000002
Length 381
CURRENT: 68.51000000000002
Length 381
CURRENT: 68.51000000000002
2018-01-05
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
Request Limit is reached
START 2018-01-05 00:00:00
Length 380
CURRENT: 68.51000000000002
Length 37

CURRENT: 143.27999999999994
Length 377
CURRENT: 143.27999999999994
Length 381
CURRENT: 143.27999999999994
Length 381
CURRENT: 143.27999999999994
Length 378
CURRENT: 143.27999999999994
Length 381
CURRENT: 143.27999999999994
Length 380
CURRENT: 143.27999999999994
Length 367
A014190 BUY1 2018-01-09 09:46:00 2015 RISK 2.81
A014190 SELL 2018-01-09 11:27:00 2025
CURRENT: 143.77999999999994
Length 371
CURRENT: 143.77999999999994
Length 380
CURRENT: 143.77999999999994
Length 381
CURRENT: 143.77999999999994
Length 381
A035600 BUY1 2018-01-09 11:24:00 17550 RISK 2.03
A035600 SELL 2018-01-09 11:34:00 17450
A035600 BUY1 2018-01-09 11:50:00 17450 RISK 1.74
A035600 SELL 2018-01-09 13:04:00 17400
CURRENT: 142.91999999999996
Length 381
CURRENT: 142.91999999999996
Length 381
CURRENT: 142.91999999999996
Length 378
CURRENT: 142.91999999999996
Length 380
CURRENT: 142.91999999999996
Length 381
CURRENT: 142.91999999999996
Length 381
CURRENT: 142.91999999999996
Length 378
CURRENT: 142.91999999999996
Length 3

CURRENT: 152.61999999999998
Length 371
CURRENT: 152.61999999999998
Length 381
A215600 BUY1 2018-01-12 13:01:00 102000 RISK 2.1
A215600 SELL 2018-01-12 13:28:00 101300
CURRENT: 151.92999999999998
Length 375
CURRENT: 151.92999999999998
Length 381
A058820 BUY1 2018-01-12 11:37:00 7028 RISK 1.93
A058820 SELL 2018-01-12 13:33:00 7028
CURRENT: 151.92999999999998
Length 381
CURRENT: 151.92999999999998
Length 381
A085660 BUY1 2018-01-12 11:05:00 37450 RISK 6.21
A085660 SELL 2018-01-12 11:15:00 37200
CURRENT: 151.26
Length 381
A095700 BUY1 2018-01-12 10:04:00 86700 RISK 5.16
A095700 SELL 2018-01-12 10:29:00 86000
CURRENT: 150.45
Length 381
CURRENT: 150.45
Length 378
CURRENT: 150.45
Length 381
CURRENT: 150.45
Length 375
CURRENT: 150.45
Length 378
CURRENT: 150.45
Length 381
CURRENT: 150.45
Length 381
CURRENT: 150.45
Length 377
CURRENT: 150.45
Length 381
A086900 BUY1 2018-01-12 10:05:00 504594 RISK 3.32
A086900 SELL 2018-01-12 10:32:00 504497
CURRENT: 150.42999999999998
Length 376
CURRENT: 150.429

A090710 SELL 2018-01-16 11:39:00 2605
CURRENT: 194.72
Length 381
CURRENT: 194.72
Length 381
CURRENT: 194.72
Length 380
CURRENT: 194.72
Length 381
CURRENT: 194.72
Length 379
A032980 BUY1 2018-01-16 10:11:00 2015 RISK 5.97
A032980 SELL 2018-01-16 11:28:00 2025
CURRENT: 195.22
Length 380
A078130 BUY1 2018-01-16 11:55:00 1000 RISK 5.76
A078130 SELL 2018-01-16 12:07:00 995
A078130 BUY1 2018-01-16 12:09:00 1015 RISK 6.81
A078130 SELL 2018-01-16 12:37:00 1000
CURRENT: 193.24
Length 381
A178920 BUY1 2018-01-16 09:50:00 46550 RISK 3.91
A178920 SELL 48400
CURRENT: 197.21
Length 381
CURRENT: 197.21
Length 381
A263750 BUY1 2018-01-16 14:18:00 262800 RISK 1.43
A263750 SELL 262900
CURRENT: 197.25
Length 380
A204840 BUY1 2018-01-16 09:54:00 4805 RISK 22.69
A204840 SELL 2018-01-16 10:28:00 4706
CURRENT: 195.19
Length 378
CURRENT: 195.19
Length 381
CURRENT: 195.19
Length 380
CURRENT: 195.19
Length 381
CURRENT: 195.19
Length 378
A052260 BUY1 2018-01-16 09:59:00 23100 RISK 7.19
A052260 SELL 2018-01-16 11

MemoryError: In RendererAgg: Out of memory

MemoryError: In RendererAgg: Out of memory

<Figure size 1080x1440 with 2 Axes>

In [ ]:
from datetime import date
from morning.back_data.holidays import is_holidays
from morning_main.trend_record.kosdaq_trend import KosdaqTrend
from_datetime = datetime(2018, 1, 1)
profits2 = []
profit_records2 = []
while from_datetime < datetime(2019, 12, 7):
    if is_holidays(from_datetime):
        from_datetime += timedelta(days = 1)
        continue
        
    kt = KosdaqTrend(from_datetime.date())
        if not kt.current_greater_than_mean():
            from_datetime += timedelta(days = 1)
            continue
        
    kbc = KosdaqSearchBullChooser(from_datetime.date())
    if len(kbc.codes) == 0:
        from_datetime += timedelta(days = 1)
        continue
        
    
    print('START', from_datetime)
    codes = kbc.codes

    for code in codes:
        p = handle(code, from_datetime.date())
        profits2.extend(p)
        print('CURRENT:', sum(profits2))
        if len(p) > 0:
            profit_records2.append((from_date, code, p))
    from_datetime += timedelta(days = 1)